# 03: Interactive Simulation Demo

This notebook demonstrates the trained agent navigating through a maze using only acoustic signals.

## Demo Features

1. Load trained model
2. Generate test maze
3. Run agent navigation using acoustic perception
4. Visualize trajectory and compare with optimal path
5. Analyze performance metrics

## How It Works

At each step:
1. Agent emits sound pulse
2. k-Wave simulates acoustic propagation and reflections
3. Microphone array captures reverberations
4. CNN processes spectrogram and predicts action
5. Agent moves based on prediction

In [ ]:
# Add src to path
import sys
sys.path.append('../')

import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from IPython.display import clear_output
import time

from src.simulation import AcousticSimulator
from src.environment import MazeGenerator, Oracle, Action
from src.model import AudioNavCNN
from src.utils import (
    plot_maze,
    plot_navigation_episode,
    plot_multi_channel_spectrogram,
    create_animation_frames
)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Load Trained Model

In [ ]:
# Model path
MODEL_PATH = '../data/audio_nav_model.pth'

# Load checkpoint
checkpoint = torch.load(MODEL_PATH, map_location=device)

# Extract configuration
model_config = checkpoint['model_config']
normalization = checkpoint['normalization']
performance = checkpoint['performance']

print("Model Configuration:")
print(f"  Microphones: {model_config['num_microphones']}")
print(f"  Actions: {model_config['num_actions']}")
print(f"  Dropout: {model_config['dropout_rate']}")

print("\nNormalization:")
print(f"  Mean: {normalization['mean']:.6f}")
print(f"  Std: {normalization['std']:.6f}")

print("\nTraining Performance:")
print(f"  Best epoch: {performance['best_epoch']}")
print(f"  Validation accuracy: {performance['final_val_accuracy']:.2f}%")

# Create and load model
model = AudioNavCNN(
    num_microphones=model_config['num_microphones'],
    num_actions=model_config['num_actions'],
    dropout_rate=model_config['dropout_rate']
).to(device)

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print("\n✓ Model loaded successfully")

## 2. Setup Test Environment

In [ ]:
# Environment configuration
MAZE_WIDTH = 20
MAZE_HEIGHT = 20
RANDOM_SEED = 123  # Different from training

# Acoustic parameters (should match training)
GRID_SPACING = 0.01
SIMULATION_DURATION = 0.015
SOURCE_FREQUENCY = 5000.0
NUM_MICROPHONES = 8

# Navigation parameters
MAX_STEPS = 100  # Maximum steps before termination

print("Test Environment Configuration:")
print(f"  Maze size: {MAZE_WIDTH}×{MAZE_HEIGHT}")
print(f"  Max steps: {MAX_STEPS}")
print(f"  Random seed: {RANDOM_SEED}")

## 3. Generate Test Maze

In [ ]:
# Generate maze
maze_gen = MazeGenerator(MAZE_WIDTH, MAZE_HEIGHT, random_seed=RANDOM_SEED)
maze = maze_gen.generate_simple_maze(wall_probability=0.3)

print(f"Maze shape: {maze.shape}")
print(f"Walkable cells: {(maze == 0).sum()}")

# Initialize oracle (for comparison)
oracle = Oracle(maze)

# Select random start and goal positions
start_pos = maze_gen.get_random_walkable_position(maze)
goal_pos = maze_gen.get_random_walkable_position(maze)

# Ensure goal is reachable from start
while not oracle.is_reachable(start_pos, goal_pos) or start_pos == goal_pos:
    goal_pos = maze_gen.get_random_walkable_position(maze)

# Fixed sound source position
source_pos = maze_gen.get_random_walkable_position(maze)

print(f"\nStart position: {start_pos}")
print(f"Goal position: {goal_pos}")
print(f"Source position: {source_pos}")

# Compute optimal path
optimal_path = oracle.find_path(start_pos, goal_pos)
print(f"Optimal path length: {len(optimal_path)} steps")

# Visualize
fig = plot_maze(
    maze,
    agent_pos=start_pos,
    goal_pos=goal_pos,
    source_pos=source_pos,
    path=optimal_path,
    title="Test Maze with Optimal Path"
)
plt.show()

## 4. Initialize Acoustic Simulator

In [ ]:
# Create simulator
simulator = AcousticSimulator(
    grid_spacing=GRID_SPACING,
    simulation_duration=SIMULATION_DURATION,
    source_frequency=SOURCE_FREQUENCY,
    num_microphones=NUM_MICROPHONES,
    pml_size=10,
)

print("✓ Acoustic simulator initialized")
print(f"  Grid spacing: {GRID_SPACING} m")
print(f"  Simulation duration: {SIMULATION_DURATION*1000} ms")
print(f"  Source frequency: {SOURCE_FREQUENCY} Hz")

## 5. Run Navigation Episode

The agent will navigate from start to goal using only acoustic perception.

In [ ]:
# Action names
action_names = ['STOP', 'UP', 'DOWN', 'LEFT', 'RIGHT']
action_deltas = {
    Action.STOP: (0, 0),
    Action.UP: (-1, 0),
    Action.DOWN: (1, 0),
    Action.LEFT: (0, -1),
    Action.RIGHT: (0, 1),
}

# Initialize navigation
current_pos = start_pos
trajectory = [current_pos]
actions_taken = []
spectrograms_history = []

reached_goal = False
step = 0

print("Starting navigation...\n")
print("⚠️ Each step requires a k-Wave simulation (~30-60 seconds)")
print("="*60)

# Navigation loop
for step in range(MAX_STEPS):
    print(f"\nStep {step + 1}/{MAX_STEPS}")
    print(f"  Current position: {current_pos}")
    
    # Check if goal reached
    if current_pos == goal_pos:
        print("  ✓ GOAL REACHED!")
        reached_goal = True
        break
    
    # Run acoustic simulation
    print("  Running k-Wave simulation...")
    sensor_data = simulator.run_simulation(
        maze=maze,
        agent_pos=current_pos,
        source_pos=source_pos,
        verbose=False
    )
    
    # Compute spectrogram
    spectrogram = simulator.compute_spectrogram(sensor_data)
    spectrograms_history.append(spectrogram)
    
    # Normalize using training statistics
    spectrogram_normalized = (spectrogram - normalization['mean']) / normalization['std']
    
    # Predict action
    with torch.no_grad():
        spec_tensor = torch.from_numpy(spectrogram_normalized).unsqueeze(0).to(device)
        action_logits = model(spec_tensor)
        action_probs = torch.softmax(action_logits, dim=1)
        predicted_action = torch.argmax(action_logits, dim=1).item()
    
    # Get optimal action for comparison
    optimal_action = oracle.get_optimal_action(current_pos, goal_pos)
    
    print(f"  Predicted action: {action_names[predicted_action]}")
    print(f"  Optimal action: {action_names[optimal_action]}")
    print(f"  Action probabilities: {action_probs[0].cpu().numpy()}")
    
    # Take action
    actions_taken.append(predicted_action)
    delta = action_deltas[Action(predicted_action)]
    next_pos = (current_pos[0] + delta[0], current_pos[1] + delta[1])
    
    # Check if move is valid
    if (
        0 <= next_pos[0] < maze.shape[0] and
        0 <= next_pos[1] < maze.shape[1] and
        maze[next_pos[0], next_pos[1]] == 0
    ):
        current_pos = next_pos
        trajectory.append(current_pos)
        print(f"  → Moved to: {current_pos}")
    else:
        print(f"  × Invalid move! Staying at: {current_pos}")
        trajectory.append(current_pos)  # Stay in place
    
    print("-" * 60)

# Final status
print("\n" + "="*60)
if reached_goal:
    print("✓ NAVIGATION SUCCESSFUL!")
    print(f"  Steps taken: {len(trajectory)}")
    print(f"  Optimal steps: {len(optimal_path)}")
    print(f"  Efficiency: {100 * len(optimal_path) / len(trajectory):.1f}%")
else:
    print("× Navigation failed (max steps reached)")
    print(f"  Steps taken: {len(trajectory)}")
    print(f"  Distance to goal: {oracle._heuristic(current_pos, goal_pos):.1f}")
print("="*60)

## 6. Visualize Navigation Episode

In [ ]:
# Plot complete trajectory
fig = plot_navigation_episode(
    maze=maze,
    trajectory=trajectory,
    goal_pos=goal_pos,
    actions=actions_taken,
    title="Agent Navigation Using Acoustic Perception"
)
plt.show()

# Plot comparison with optimal path
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Optimal path
plot_maze(
    maze,
    agent_pos=start_pos,
    goal_pos=goal_pos,
    path=optimal_path,
    title=f"Optimal Path (A*): {len(optimal_path)} steps",
    ax=ax1
)

# Agent path
plot_maze(
    maze,
    agent_pos=start_pos,
    goal_pos=goal_pos,
    path=trajectory,
    title=f"Agent Path (Acoustic): {len(trajectory)} steps",
    ax=ax2
)

plt.tight_layout()
plt.show()

## 7. Analyze Action Predictions

In [ ]:
# Compare predicted vs optimal actions
optimal_actions_for_trajectory = []
for pos in trajectory[:-1]:  # Exclude final position
    optimal_actions_for_trajectory.append(
        oracle.get_optimal_action(pos, goal_pos)
    )

# Compute accuracy
actions_array = np.array(actions_taken)
optimal_array = np.array(optimal_actions_for_trajectory)
action_accuracy = 100 * (actions_array == optimal_array).sum() / len(actions_array)

print(f"Action Prediction Analysis:")
print(f"  Total actions: {len(actions_taken)}")
print(f"  Correct actions: {(actions_array == optimal_array).sum()}")
print(f"  Accuracy: {action_accuracy:.2f}%")

# Action distribution
print(f"\nAction Distribution:")
for action_idx, action_name in enumerate(action_names):
    count = (actions_array == action_idx).sum()
    print(f"  {action_name}: {count} ({100*count/len(actions_array):.1f}%)")

# Plot action comparison
fig, ax = plt.subplots(figsize=(12, 4))
x = np.arange(len(actions_taken))
ax.plot(x, actions_taken, 'o-', label='Predicted', linewidth=2, markersize=6)
ax.plot(x, optimal_actions_for_trajectory, 's--', label='Optimal', linewidth=2, markersize=6, alpha=0.7)
ax.set_xlabel('Step')
ax.set_ylabel('Action')
ax.set_yticks(range(5))
ax.set_yticklabels(action_names)
ax.set_title('Action Predictions vs Optimal Actions')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Visualize Sample Spectrograms

In [ ]:
# Show spectrograms from first few steps
num_samples = min(3, len(spectrograms_history))

for i in range(num_samples):
    print(f"\nStep {i+1}:")
    print(f"  Position: {trajectory[i]}")
    print(f"  Action: {action_names[actions_taken[i]]}")
    
    fig = plot_multi_channel_spectrogram(
        spectrograms_history[i],
        title=f"Step {i+1} Spectrogram (Position: {trajectory[i]}, Action: {action_names[actions_taken[i]]})"
    )
    plt.show()

## 9. Performance Metrics Summary

In [ ]:
# Compute metrics
if reached_goal:
    path_efficiency = 100 * len(optimal_path) / len(trajectory)
    extra_steps = len(trajectory) - len(optimal_path)
else:
    path_efficiency = 0.0
    extra_steps = float('inf')
    final_distance = oracle._heuristic(current_pos, goal_pos)

# Summary table
print("\n" + "="*60)
print("NAVIGATION PERFORMANCE SUMMARY")
print("="*60)
print(f"\nEnvironment:")
print(f"  Maze size: {MAZE_WIDTH}×{MAZE_HEIGHT}")
print(f"  Start: {start_pos}")
print(f"  Goal: {goal_pos}")
print(f"  Optimal path length: {len(optimal_path)} steps")

print(f"\nAgent Performance:")
print(f"  Goal reached: {'✓ Yes' if reached_goal else '× No'}")
print(f"  Steps taken: {len(trajectory)}")
if reached_goal:
    print(f"  Extra steps: {extra_steps}")
    print(f"  Path efficiency: {path_efficiency:.1f}%")
else:
    print(f"  Final distance to goal: {final_distance:.1f}")
print(f"  Action accuracy: {action_accuracy:.1f}%")

print(f"\nModel Information:")
print(f"  Training accuracy: {performance['final_val_accuracy']:.2f}%")
print(f"  Best epoch: {performance['best_epoch']}")

print("\n" + "="*60)

## 10. Optional: Create Animation Frames

Generate individual frames that can be combined into a video.

In [ ]:
# Uncomment to create animation frames
# create_animation_frames(
#     maze=maze,
#     trajectory=trajectory,
#     goal_pos=goal_pos,
#     output_dir='../data/animation_frames',
#     figsize=(8, 8)
# )
# 
# print("\n✓ Animation frames created!")
# print("To create video, run:")
# print("  ffmpeg -framerate 5 -i ../data/animation_frames/frame_%04d.png -c:v libx264 -pix_fmt yuv420p navigation.mp4")

print("\n✓ Demo complete!")
print("\nExperiment with:")
print("  - Different maze configurations")
print("  - Various start/goal positions")
print("  - Modified acoustic parameters")
print("  - Alternative model architectures (AudioNavResNet)")